# Training Model with V1 Datas

### Initializing GPU

In [1]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)


2025-07-21 11:34:21.855698: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753090461.960557    2979 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753090461.994660    2979 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1753090462.164489    2979 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753090462.164568    2979 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1753090462.164572    2979 computation_placer.cc:177] computation placer alr

### Imports

In [ ]:

from tensorflow import keras
import numpy as np
import json
from sklearn.model_selection import train_test_split
import os
import  random
import numpy as np
from tensorflow.keras import layers
import keras as keras
from keras.applications import  Xception
from PIL import Image
import random
print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.config.threading.set_intra_op_parallelism_threads(4)
tf.config.threading.set_inter_op_parallelism_threads(4)
np.set_printoptions(precision=3, suppress=True)



2.19.0
2.19.0
Num GPUs Available:  1


### Global variables

In [ ]:

IS_WSL = True
WINDOWS_DATA_FOLDER="/mnt/e/code/ai/data/tcg/"
GENERATED_DATAS_V1_FOLDER_PATH = "data/generated_datas_v1/"
if IS_WSL :
    GENERATED_DATAS_V1_FOLDER_PATH = WINDOWS_DATA_FOLDER + GENERATED_DATAS_V1_FOLDER_PATH

JSONS_FOLDER=GENERATED_DATAS_V1_FOLDER_PATH+"json/"
IMAGES_FOLDER=GENERATED_DATAS_V1_FOLDER_PATH+"images/"

RANDOM_STATE=42
RESIZE_SIZE=224
RGB = True
EPOCHS = 10
BATCH_SIZE = 64
TEST_SIZE = 0.2
MIN_ASSETS=0
MAX_ASSETS=30000
if RGB:
    RGB_VALUE = 3
else:
    RGB_VALUE = 1



In [3]:

def load_files_list():

    json_files_list = []
    for json_file in os.listdir(JSONS_FOLDER):
        if json_file.endswith(".json"):
            json_files_list.append(json_file)
           

    random.Random(RANDOM_STATE).shuffle(json_files_list)
    json_files_list = json_files_list[MIN_ASSETS:MAX_ASSETS]
    print("Total files : ",json_files_list.__len__())
    print("First file : ",json_files_list[0])
    return json_files_list



### Loading files in generator

In [4]:
def read_json_from_s3(json_key):
    with open(JSONS_FOLDER + json_key) as json_data:
        raw_json = json.load(json_data)
        image_key = IMAGES_FOLDER + raw_json["asset"]["name"]
        points = raw_json["regions"][0]["points"]
        x0 = points[0]["x"]
        y0 = points[0]["y"] 
        x1 = points[1]["x"] 
        y1 = points[1]["y"] 
        x2 = points[2]["x"]
        y2 = points[2]["y"]
        x3 = points[3]["x"]
        y3 = points[3]["y"]
        new_json = {
            "image_key": image_key,
            "bounding_boxes": [x0, y0, x1, y1, x2, y2, x3, y3]
        }
        return new_json
    return None
# Fonction pour lire les images depuis S3
def read_image_from_s3(image_key):
    try:
        img = Image.open(image_key)
        img = img.resize((RESIZE_SIZE, RESIZE_SIZE))
        img = np.asarray( img, dtype="int32" )
        np.array(img)
        img = img/255
        return img
    except Exception as e:
        print(e)
        return None
# Fonction pour créer un dataset TensorFlow
def create_dataset(json_keys, batch_size=32):
    def generator():
        for json_key in json_keys:
            annotations = read_json_from_s3(json_key)
            if annotations == None or annotations["image_key"] == None :
                print("File : " + json_key + " is Malformated, delete it")
                continue
            img = read_image_from_s3(annotations["image_key"])
            if img is None:
                print("Image : " + annotations["image_key"] + " is Malformated, delete it")
                continue
            yield img, annotations['bounding_boxes']
    dataset = tf.data.Dataset.from_generator(generator,
                                             output_signature=(
                                                 tf.TensorSpec(shape=(RESIZE_SIZE, RESIZE_SIZE, RGB_VALUE), dtype=tf.float32),
                                                 tf.TensorSpec(shape=(8,), dtype=tf.float32)
                                             ))
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    return dataset

### Load datasets

In [ ]:
json_files_list = load_files_list()

train_files, val_files = train_test_split(json_files_list, shuffle=True, random_state=42, test_size=TEST_SIZE)
print("Train files : ", len(train_files))
print("Val files : ", len(val_files))
train_dataset = create_dataset(train_files, BATCH_SIZE)
val_dataset = create_dataset(val_files, BATCH_SIZE)


Total files :  65
First file :  base1-36_-10_-7_-4_32_614.json
Train files :  52
Val files :  13


I0000 00:00:1753090499.392334    2979 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6704 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1070, pci bus id: 0000:01:00.0, compute capability: 6.1


In [6]:

# Définir explicitement la couche d'entrée
input_layer = layers.Input(shape=(RESIZE_SIZE, RESIZE_SIZE, RGB_VALUE))
xception = Xception(weights="imagenet", include_top=False)(input_layer)

# Construire le reste du modèle
flatten = layers.Flatten(name='flatten')(xception)
locator_branch = layers.Dense(128, activation='relu', name='bb_1')(flatten)
locator_branch = layers.Dense(64, activation='relu', name='bb_2')(locator_branch)
locator_branch = layers.Dense(32, activation='relu', name='bb_3')(locator_branch)
locator_branch = layers.Dense(8, activation='sigmoid', name='bb_head')(locator_branch)

# Créer le modèle
model = tf.keras.Model(inputs=input_layer, outputs=[locator_branch])

# Résumé du modèle
model.summary()

# Définir les pertes et compiler le modèle
losses = {"bb_head": tf.keras.losses.MeanSquaredError()}


model.compile(loss=losses, optimizer='Adam', metrics=['accuracy'])




Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ xception (Functional)           │ (None, 7, 7, 2048)     │    20,861,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 100352)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bb_1 (Dense)                    │ (None, 128)            │    12,845,184 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bb_2 (Dense)                    │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bb_3 (Dense)                    │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bb_head (Dense)                 │ (None, 8)              │           264 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 33,717,264 (128.62 MB)

 Trainable params: 33,662,736 (128.41 MB)

 Non-trainable params: 54,528 (213.00 KB)

In [7]:
# Callbacks
checkpoint = keras.callbacks.ModelCheckpoint(
    filepath='checkpoints/model_checkpoint_{epoch:02d}.keras',
    save_best_only=True,
    monitor='val_accuracy',
    mode='max',
    save_weights_only=False,
    verbose=1
)

early_stopping = keras.callbacks.EarlyStopping(
    monitor='loss',
    patience=3,
    mode='min',
    verbose=1
)

In [8]:

# Entraînement du modèle
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=EPOCHS,
    callbacks=[checkpoint,early_stopping]
)


Epoch 1/8


I0000 00:00:1753090535.928339    3028 service.cc:152] XLA service 0x7896000048a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753090535.928453    3028 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce GTX 1070, Compute Capability 6.1
2025-07-21 11:35:36.485372: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1753090539.729314    3028 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-07-21 11:35:44.376088: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:382] Garbage collection: deallocate free memory regions (i.e., allocations) so that we can re-allocate a larger region to avoid OOM due to memory fragmentation. If you see this message frequently, you are running near the threshold of the available device memory and re-allocation may incur great performance overhead. You may try smaller batch size

      1/Unknown 79s 79s/step - accuracy: 0.0962 - loss: 0.1014

2025-07-21 11:36:28.171170: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2025-07-21 11:36:28.171876: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]
/home/guillianv/miniconda3/envs/tf/lib/python3.12/site-packages/keras/src/trainers/epoch_iterator.py:160: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()



Epoch 1: val_accuracy improved from -inf to 0.15385, saving model to checkpoints/model_checkpoint_01.keras


...

2025-07-21 11:37:10.730171: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.3077 - loss: 0.0810
Epoch 5: val_accuracy did not improve from 0.46154
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step - accuracy: 0.3077 - loss: 0.0810 - val_accuracy: 0.4615 - val_loss: 0.0811
Epoch 6/8


2025-07-21 11:37:19.325200: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 10810207217113351831
2025-07-21 11:37:19.325247: I tensorflow/core/framework/local_rendezvous.cc:426] Local rendezvous recv item cancelled. Key hash: 14185708451129695356


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.3077 - loss: 0.0807
Epoch 6: val_accuracy did not improve from 0.46154
1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step - accuracy: 0.3077 - loss: 0.0807 - val_accuracy: 0.4615 - val_loss: 0.0850
Epoch 7/8
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.3077 - loss: 0.0805
Epoch 7: val_accuracy did not improve from 0.46154
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step - accuracy: 0.3077 - loss: 0.0805 - val_accuracy: 0.4615 - val_loss: 0.0854
Epoch 8/8
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.3077 - loss: 0.0802
Epoch 8: val_accuracy did not improve from 0.46154
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step - accuracy: 0.3077 - loss: 0.0802 - val_accuracy: 0.4615 - val_loss: 0.0853


2025-07-21 11:37:44.807556: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[IteratorGetNext/_2]]


In [21]:
model.save("model-28k.keras")